In [6]:
%load_ext autoreload
%autoreload 2
import boto3
import tqdm

import json
from pathlib import Path
import gzip

import numpy as np
import pandas as pd

import re

from itertools import chain

from datetime import datetime

from bloom_filter import BloomFilter

from urllib.parse import urlparse

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
!pip install boto3 pandas tqdm

     |████████████████████████████████| 69 kB 1.4 MB/s eta 0:00:011


# In our middleware, this is how urls are preprocessed before being added.

In [15]:
def urlpath(url): return urlparse(url).path

In [18]:
startpage_links = [
    "https://www.dr.dk/nyheder/indland",
    "https://www.dr.dk/nyheder/udland",
    "https://www.dr.dk/nyheder/penge",
    "https://www.dr.dk/nyheder/politik",
    "https://www.dr.dk/nyheder/regionale",
    "https://www.dr.dk/sporten",
    "https://www.dr.dk/nyheder/kultur",
    "https://www.dr.dk/nyheder/vejret",
    "https://www.dr.dk/nyheder/viden",
    "https://www.dr.dk/nyheder",
]
spl = set(startpage_links)

In [8]:
%%time
lines = []
with open("dr_frontpage_total.jsonl") as reader:
    for line in reader:
        lines.append(json.loads(line))

CPU times: user 2.61 s, sys: 249 ms, total: 2.85 s
Wall time: 2.86 s


In [16]:
urls = set(urlpath(line.get("url")[0]) for line in lines)

In [23]:
urls & spl

set()

In [25]:
filterr = BloomFilter(max_elements=4000000, error_rate=1e-9, filename="dr_frontpage.filter")
for url in tqdm.tqdm(urls):
    filterr.add(url)
filterr.backend.close()

100%|██████████| 113760/113760 [00:23<00:00, 4788.05it/s]
